In [1]:
import os
import requests
from io import BytesIO
from PIL import Image
from api_key import API_KEY

In [2]:
def get_streetview_images(latitude, longitude, output_directory='images', angles=[0, 90, 180, 270]):
    base_url = "https://maps.googleapis.com/maps/api/streetview"
    
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    image_list = []
    
    for angle in angles:
        params = {
            'size': '600x600',
            'location': f'{latitude},{longitude}',
            'heading': angle,
            'key': API_KEY
        }
        
        response = requests.get(base_url, params=params)
        
        if response.status_code == 200:
            image = Image.open(BytesIO(response.content))
            image_list.append(image)
            
            output_file = os.path.join(output_directory, f'{latitude}_{longitude}_{angle}.jpg')
            image.save(output_file)
            
            print(f'Image saved as: {output_file}')
        else:
            print(f'Error retriving angle {angle}. Status {response.status_code}')
    
    return image_list

In [3]:
#get_streetview_images(-16.7302267, -47.8860483)